In [ ]:
#cleaning the data set
## Dataset from: https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db

import pandas as pd
df = pd.read_csv("/home/guna/Downloads/SpotifyFeatures.csv")
df=df.drop(columns=['track_id','duration_ms','key','mode','time_signature'])
df=df.dropna()
df=df.reset_index(drop=True)

In [ ]:
df.info()

In [99]:
df.shape

(232724, 13)

In [30]:
df.head()

,genre,artist_name,track_name,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0,0.611,0.389,0.910,0.000,0.3460,-1.828,0.0525,166.969,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),1,0.246,0.590,0.737,0.000,0.1510,-5.559,0.0868,174.003,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,3,0.952,0.663,0.131,0.000,0.1030,-13.879,0.0362,99.488,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0,0.703,0.240,0.326,0.000,0.0985,-12.178,0.0395,171.758,0.227
4,Movie,Fabien Nataf,Ouverture,4,0.950,0.331,0.225,0.123,0.2020,-21.150,0.0456,140.576,0.390


In [31]:
meta_col=['genre','artist_name','track_name']
feature_col=['acousticness','danceability','energy','instrumentalness','liveness',
             'loudness','speechiness','tempo','valence']
df_meta=df[meta_col]
df_feature = df[feature_col]

In [32]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(df_feature)


In [33]:
#Step 8: Final check
print("🎧 Total songs:", df.shape[0])


🎧 Total songs: 232724


In [46]:
from sklearn.neighbors import NearestNeighbors

In [84]:
def recommend_song(track, artist=None, top=5, filter_genre=True, skip_weird_genres=True):
    knn_model = NearestNeighbors(n_neighbors=top + 10, metric='cosine')
    knn_model.fit(X_scaled)
    if artist:
        mask = (
            (df_meta['track_name'].str.lower() == track.lower()) &
            (df_meta['artist_name'].str.lower() == artist.lower())
        )
    else:
        mask = (df_meta['track_name'].str.lower() == track.lower())

    idk_list = df_meta[mask].index.tolist()
    if not idk_list:
        print(f"❌ No song named '{track}' was found.")
        return
    idk = idk_list[0]
    query = X_scaled[idk].reshape(1, -1)
    dist, ind = knn_model.kneighbors(query)
    original_song = df_meta.iloc[idk]
    original_genre = original_song['genre']
    print(f"\n🎧 Songs similar to '{original_song['track_name']}' by {original_song['artist_name']}:\n")
    shown = 0
    for i in ind[0][1:]:
        song = df_meta.iloc[i]
        if skip_weird_genres and song['genre'].lower() in ['children’s music', "children's music"]:
            continue
        if filter_genre and song['genre'] != original_genre:
            continue
        print(f"✔️ {song['track_name']} — {song['artist_name']} ({song['genre']})")
        shown += 1
        if shown >= top:
            break
    if shown == 0:
        print(" No similar songs found with matching genre. Try turning off filtering.")

In [80]:
recommend_song('perfect','ed sheeran')


🎧 Songs similar to 'Perfect' by Ed Sheeran:

✔️ HeartLess — Madison Beer (Pop)
✔️ Goner — Twenty One Pilots (Pop)


In [81]:
recommend_song('Heartless','madison beer')


🎧 Songs similar to 'HeartLess' by Madison Beer:

✔️ HeartLess — Madison Beer (Dance)
✔️ Teardrops On My Guitar - Radio Single Remix — Taylor Swift (Dance)
✔️ SHE DON'T LOVE ME — ZAYN (Dance)


In [82]:
recommend_song('Teardrops On My Guitar - Radio Single Remix','Taylor Swift')


🎧 Songs similar to 'Teardrops On My Guitar - Radio Single Remix' by Taylor Swift:

✔️ (Another Song) All Over Again — Justin Timberlake (Dance)
✔️ HeartLess — Madison Beer (Dance)
✔️ Na Na — Trey Songz (Dance)
